In [95]:
import pprint
import re
import csv
import json
from bs4 import BeautifulSoup

In [96]:
pp = pprint.PrettyPrinter(indent=4)

In [102]:
testif = re.compile(r'^testif.{0,35}$', re.I | re.M)
adjourned = re.compile(r'.*adjourned', re.I | re.M)
gavel = re.compile(r'.*gavel at', re.I | re.M)
repres = re.compile(r'representative.{0,15}testified on .{0,15}$', re.I | re.M)
bef = re.compile(r'.*before the committee.$', re.I | re.M)
mot = re.compile(r'.*motion {1,5}prevailed.{0,1}$', re.I | re.M)
adop = re.compile(r'.*adopted.{0,1}$', re.I | re.M)
amen = re.compile(r'.*moved amendment.{0,2}$', re.I | re.M)
moved = re.compile(r'^.{0,30}move[ds]{0,1}', re.I | re.M)
ton = re.compile(r'^.{0,25}testified [oi]{1}n.{0,30}$', re.I | re.M)
dis = re.compile(r'^.{0,4}discussion.*$', re.I | re.M)
numb = re.compile(r'^[A-Z0-9-]{0,10}$', re.I | re.M)
und = re.compile(r'^_+$', re.I | re.M)
quo = re.compile(r'^a quorum was present.{0,15}$', re.I | re.M)
prs = re.compile(r'^.{0,35}presented.{0,45}$', re.I | re.M)
vot = re.compile(r'^.*voted .{0,6}:$', re.I | re.M)
tho = re.compile(r'^those testifying on the bill.*:$', re.I | re.M)
amd = re.compile(r'^.{0,3}amendment.{0,3}$', re.I | re.M)
pag = re.compile(r'^.{0,2}page \d{0,3}', re.I | re.M)
omn = re.compile(r'.*omnibus bill.{0,3}$', re.I | re.M)
conv = re.compile(r'^the committee reconvened at .{0,40}$', re.I | re.M)
rec = re.compile(r'^the committee recessed at .{0,40}$', re.I | re.M)
mins = re.compile(r'^minutes [ofr]{2,3} .{5,40}\d$', re.I | re.M)
merec = re.compile(r'^.{0,5}meeting was recessed at .{3,15}$', re.I | re.M)
nub = re.compile(r'^[A-Z0-9-]{4,11} \(.{4,15}\)', re.I | re.M)

In [107]:
with open('mncsv.csv', newline='') as csvfile:
    rows = list(csv.DictReader(csvfile))[6001:] # slice the rows here
    with open('minnesota6001-7000.json', 'w') as jsonfile: # change output file name here
        for row in rows:
            soup = BeautifulSoup(row.get('html'), 'html5lib')
            text = soup.get_text('\n')
            lines_list = []
            lines = enumerate(text.split('\n'))
            ls, le = 0, -1
            for lnNum, line in lines:
                le = len(line) + le
                lines_list.append((lnNum, ls, le, line, 'testif' in line.lower()))
                ls = le + 1
            
            smell_list = []
            for lnNum, start_char, end_char, name, containsTestif in lines_list:
                if containsTestif:
                    offset = lnNum + 8 if lnNum + 8 <= lines_list[-1][0] else lines_list[-1][0] + 1
                    smell_list += lines_list[lnNum:offset] # current and next 7 lines (total 8 lines)
                    
            orgs = [{'name': smell[3], 'start_char': smell[1], 'end_char': smell[2], 
                                     'position': None} for smell in smell_list]
            
            orgs = [dict(t) for t in set([tuple(d.items()) for d in orgs])] # remove dict duplicates
            
            # it's possible to get bill_name and stance (position) from below 
            orgs = [d for d in orgs if not testif.match(d['name'].strip().lower()) \
                                   and not adjourned.match(d['name'].strip().lower()) \
                                   and not gavel.match(d['name'].strip().lower()) \
                                   and not repres.match(d['name'].strip().lower()) \
                                   and not bef.match(d['name'].strip().lower()) \
                                   and not mot.match(d['name'].strip().lower()) \
                                   and not adop.match(d['name'].strip().lower()) \
                                   and not amen.match(d['name'].strip().lower()) \
                                   and not moved.match(d['name'].strip().lower()) \
                                   and not ton.match(d['name'].strip().lower()) \
                                   and not dis.match(d['name'].strip().lower()) \
                                   and not numb.match(d['name'].strip().lower()) \
                                   and not und.match(d['name'].strip().lower()) \
                                   and not quo.match(d['name'].strip().lower()) \
                                   and not prs.match(d['name'].strip().lower()) \
                                   and not vot.match(d['name'].strip().lower()) \
                                   and not tho.match(d['name'].strip().lower()) \
                                   and not amd.match(d['name'].strip().lower()) \
                                   and not pag.match(d['name'].strip().lower()) \
                                   and not omn.match(d['name'].strip().lower()) \
                                   and not conv.match(d['name'].strip().lower()) \
                                   and not rec.match(d['name'].strip().lower()) \
                                   and not mins.match(d['name'].strip().lower()) \
                                   and not merec.match(d['name'].strip().lower()) \
                                   and not nub.match(d['name'].strip().lower()) \
                    
                   ]
            row['organizations'] = orgs
            m, d, y = row.get('date').split('/')
            m, d, y = m if len(m) == 2 else '0' + m, d if len(d) == 2 else '0' + d, y
            date = y + '-' + m + '-' + d # date format YYYY-MM-DD
            row['date'] = date
            
            print(row['url'])
            
            json.dump(row, jsonfile)
            jsonfile.write('\n')
        

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=4261
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=4260
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=3972
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=3973
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=3912
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=3937
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=4026
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=3626
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=3984
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86104&ls_year=86&id=3288
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86103&ls_year=86&id=3992
http://www.house.leg.

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=3788
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4386
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4385
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4053
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4052
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4074
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4160
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4191
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=3786
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=4142
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86123&ls_year=86&id=3872
http://www.house.leg.

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=4420
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=4464
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=4421
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=4035
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86107&ls_year=86&id=3361
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=4106
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=3819
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=3308
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=4171
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86107&ls_year=86&id=3377
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86126&ls_year=86&id=4422
http://www.house.leg.

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86133&ls_year=86&id=3965
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86133&ls_year=86&id=4265
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86133&ls_year=86&id=3629
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86133&ls_year=86&id=3968
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86133&ls_year=86&id=3964
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86110&ls_year=86&id=3258
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86133&ls_year=86&id=4194
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86110&ls_year=86&id=3280
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86110&ls_year=86&id=3279
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86110&ls_year=86&id=3256
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86110&ls_year=86&id=3359
http://www.house.leg.

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86131&ls_year=86&id=4078
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86131&ls_year=86&id=4140
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86142&ls_year=86&id=4046
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86131&ls_year=86&id=3958
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86131&ls_year=86&id=3959
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86131&ls_year=86&id=4178
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86142&ls_year=86&id=4180
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86131&ls_year=86&id=3758
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86106&ls_year=86&id=3227
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86106&ls_year=86&id=3859
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86131&ls_year=86&id=4360
http://www.house.leg.

...

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=3417
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=3388
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86118&ls_year=86&id=4023
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=3520
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=4518
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86118&ls_year=86&id=3996
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=4390
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=4388
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=4440
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=4434
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=86137&ls_year=86&id=3519
http://www.house.leg.

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56854
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56702
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56904
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56927
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56893
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56738
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56793
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56944
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56971
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56632
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90019&ls_year=90&id=56876
http://www

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56751
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56785
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56761
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56807
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56831
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90007&ls_year=90&id=56727
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56860
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56720
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90007&ls_year=90&id=57101
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=57112
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90012&ls_year=90&id=56892
http://www

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90017&ls_year=90&id=57110
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56822
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56723
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56984
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56872
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56786
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56921
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90024&ls_year=90&id=56707
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56966
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=56630
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90018&ls_year=90&id=57021
http://www

http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90004&ls_year=90&id=57002
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90003&ls_year=90&id=57059
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90004&ls_year=90&id=57003
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90004&ls_year=90&id=56935
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90002&ls_year=90&id=56640
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90003&ls_year=90&id=57087
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90003&ls_year=90&id=56832
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90002&ls_year=90&id=56974
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90002&ls_year=90&id=56939
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90004&ls_year=90&id=56858
http://www.house.leg.state.mn.us/cmte/minutes/minutes.aspx?comm=90002&ls_year=90&id=56985
